In [31]:
#Data Structure 
tag_sp=["INDI", "FAM"]
tag_zero=["HEAD", "TRLR", "NOTE"]
tag_one=["NAME", "SEX", "BIRT", "DEAT","FAMC","FAMS","MARR", "DIV","HUSB","WIFE","CHIL"]
tag_fam={"INDI":["NAME", "SEX","BIRT", "DEAT","FAMC","FAMS"],
             "FAM":["MARR", "DIV","HUSB","WIFE","CHIL"], 
             "DATE":["BIRT", "DEAT", "DIV", "MARR"]}


In [32]:
def isDateP(A):
    if A[1] in tag_fam["DATE"]:
        return True
    else:
        return False

In [35]:
def read_in(file):
    doc={"INDI":[], "FAM":[]}
    dic={}
    flag=False
    with open(file) as f:
        for line in f:
            A=line.strip().split(" ")
            if len(A)==3 and A[0]=='0' and A[2]== "INDI":
                if dic:
                    doc[fam].append(dic)  
                fam="INDI"
                dic={}
                dic["INDI"]=A[1]
            elif len(A)==3 and A[0]=='0' and A[2]=="FAM":
                if dic:
                    doc[fam].append(dic) 
                fam="FAM"
                dic={}
                dic["FAM"]=A[1]
            elif (A[1]=="DATE" and flag):
                flag=False
                dic[tmp]=' '.join(A[2:])
            elif A[0]=='1' and A[1] in tag_one:
                if (isDateP(A)):
                    tmp=A[1]
                    flag=True
                else:
                    dic[A[1]]=' '.join(A[2:])
        return doc   
            
            
                
            
                

In [36]:
document= read_in("./myTest.ged")

In [44]:
if "DEAT" not in document["INDI"][3]:
    print(1)
    

1
